In [91]:
import Pkg
Pkg.activate("MMLDS_project")
#Pkg.update()
#Pkg.instantiate()
#Pkg.add("ProgressMeter")
#Pkg.add("DynamicalSystems")
#Pkg.add("ReservoirComputing")
#Pkg.add("Plots")
#Pkg.add("Printf")
#Pkg.add("DifferentialEquations")
#Pkg.add("OrdinaryDiffEq")
#Pkg.add("NetCDF")
#Pkg.add("JLD2")

using ProgressMeter
using DynamicalSystems, ReservoirComputing, Plots, Printf
using DifferentialEquations
using OrdinaryDiffEq
using NetCDF
using JLD2



  Activating project at `~/Documents/GitHub/compare_ML_DynamicalSystem/MMLDS_project/src/MMLDS_project`


In [92]:
#notebook not working without input
include("data.jl")
include("Reservoir_Computing.jl")

load_ESN (generic function with 1 method)

In [93]:
#Loading and changing El Nino Data
path_to_data = "sst.mon.mean.nc"

x = ncread(path_to_data, "sst") #sst = Sea Surface Temperature
x_reduced = x[121:170, 86:95, :] #only concerned with the important region for the El Nino happening

X_matirx = compress_data_matrix(x_reduced, 5, false, false)

#our esn accepts vectors as input
X_vector = reshape(X_matirx, (20,2040))
#print("Data successfully loaded.")

20×2040 Matrix{Float64}:
 28.0325  27.8742  28.1076  28.688   …  28.621   29.4195  29.5172  29.0083
 28.2426  28.2316  28.488   28.9117     28.3489  29.3835  29.6865  29.3908
 28.5179  28.6131  28.8473  29.1355     28.6434  29.723   29.7411  29.3795
 28.84    28.8382  29.0742  29.1841     29.4347  30.2484  29.8994  29.4668
 29.023   28.9616  29.1702  29.2372     29.4146  30.2277  29.8914  29.5535
 29.1572  29.1256  29.2598  29.2776  …  29.7037  30.4001  30.031   29.7284
 29.1264  29.117   29.196   29.2381     29.8099  30.4189  30.0984  29.8489
 28.9993  29.0324  29.076   29.1209     30.1048  30.5088  30.2152  29.9283
 28.7191  28.7542  28.6979  28.8395     30.32    30.6631  30.3864  30.0508
 28.4407  28.3725  28.2181  28.6071     30.32    30.5643  30.4019  30.1978
 28.7257  28.5573  28.5916  28.9932  …  27.2255  28.4254  29.3495  29.8018
 28.7958  28.7403  28.7715  29.102      26.6425  28.0568  29.141   29.8132
 28.8938  28.9374  28.9322  29.1185     27.5829  28.7421  29.3249  29.7972


In [94]:
#generating train, val and test set
train_data, val_data, test_data = train_val_test_split(X_vector, val_seconds = 5, test_seconds = 5)

#Define grid:
# Set up the hyperparameter grid-search
param_grid = []

reservoir_sizes = [256, 512, 1024, 2048, 4096]
spectral_radii = [0.8, 0.9, 1.0,1.1, 1.2] #1,1 gives good results
sparsities = [0.01, 0.03, 0.05]
input_scales = [0.1,0.01, 0.05]
ridge_values = [1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1]
# Take the Cartesian product of the possible values
for params in Iterators.product(reservoir_sizes, spectral_radii, sparsities, input_scales, ridge_values)
    push!(param_grid, ESNHyperparams(params...))
end
println(length(param_grid), " hyperparameter combinations.")

1350 hyperparameter combinations.


In [95]:
#train network
#@time esn, W_out = cross_validate_esn(train_data, val_data, param_grid)
esn, W_out = cross_validate_esn(train_data, val_data, param_grid)

ESNHyperparams(256, 0.8, 0.01, 0.1, 1.0e-6)
Validation loss = 4.0e+02


ESNHyperparams(512, 0.8, 0.01, 0.1, 1.0e-6)
Validation loss = 3.2e+02


Progress:   0%|▏                                        |  ETA: 0:38:49

ESNHyperparams(1024, 0.8, 0.01, 0.1, 1.0e-6)
Validation loss = 2.7e+02


Progress:   0%|▏                                        |  ETA: 1:54:51

...

Progress:  99%|████████████████████████████████████████▊|  ETA: 0:01:52

Progress: 100%|████████████████████████████████████████▉|  ETA: 0:01:36

Progress: 100%|████████████████████████████████████████▉|  ETA: 0:01:20

Progress: 100%|█████████████████████████████████████████| Time: 6:00:18


(ESN{Int64, Matrix{Float64}, Default, NLADefault, Matrix{Float64}, RNN{typeof(NNlib.tanh_fast), Float64}, Matrix{Float64}, Matrix{Float64}, StandardStates, Int64, Matrix{Float64}}(1020, [28.032480239868164 27.874191284179688 … 28.628190994262695 29.02614402770996; 28.24264144897461 28.231639862060547 … 28.107284545898438 28.52972412109375; … ; 29.0902042388916 29.012521743774414 … 29.2639217376709 29.486040115356445; 28.802597045898438 28.66204071044922 … 29.333803176879883 29.558361053466797], Default(), NLADefault(), [0.0048492054591365505 0.0 … 0.0 0.0; 0.005036785657972866 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.030993514627426402; 0.0 0.0 … 0.0 -0.03896429638512419], RNN{typeof(NNlib.tanh_fast), Float64}(NNlib.tanh_fast, 1.0), [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … -0.2437075099921008 0.0], [0.0; 0.0; … ; 0.0; 0.0;;], StandardStates(), 0, [0.1351041105646223 0.5035707674353428 … 0.6863037808444159 0.6836286713493149; 0.14026275533369673 -0.8167697599591

In [96]:
#predictions
# does not work: predictions = ESNpredict(esn, 100,W_out)
predictions = esn(Generative(100), W_out)
print("The size of the predictions is: ", size(predictions))
print("The size of the X_vector is: ", size(X_vector))

times = 1:1:100
label = ["actual" "predicted"]

for i in 1:20
    i_data = X_vector[i, 1:100]
    p = plot(times, [i_data, predictions[i,1:100]], label = label, ylabel = "temperature")#
    saveEnding = string(i)
    savePath = string("/home/tom/Documents/University/3M_semester/Project_MMLDS/MMLDS_2/MMLDS_project/Figures/El_Nino_Predictions/", saveEnding,".png" )
    savefig(p, savePath)
end
    

The size of the predictions is: (20, 100)The size of the X_vector is: (20, 2040)

In [97]:
network_pathString = "esn_network.jld2"
W_out_pathString = "W_out_matrix.jld2"
save_ESN(esn, network_pathString, W_out, W_out_pathString)
esn_loaded, W_out_loaded = load_ESN(network_pathString, W_out_pathString)

predictions_loaded = esn_loaded(Generative(100), W_out_loaded)

first_field_predicted = predictions_loaded[1,1:100]

times = 1:1:100
label = ["actual" "predicted"]

for i in 1:20
    i_data = X_vector[i, 1:100]
    p = plot(times, [i_data, predictions_loaded[i,1:100]], label = label, ylabel = "temperature")#
    saveEnding = string(i)
    savePath = string("/home/tom/Documents/University/3M_semester/Project_MMLDS/MMLDS_2/MMLDS_project/Figures/Loaded_El_Nino_Predictions/", saveEnding,".png" )
    savefig(p, savePath)
end